# **Introduction**

This ipython notebook file downloads the video game genre data from the github repo and trains on it to predict from video game images what possible class the image belongs to. 

The classes here are: FPS, RTS and SPT

FPS- First Person Shooter
RTS- Real Time Strategy
SPT- Sports

**Get Data**

Getting the data by cloning the already prepared data from github.

There is a python script available to data scrap from youtube and convert it into images.

In [41]:
!git clone https://github.com/GuruShiva/video_game_genre_data.git

Cloning into 'video_game_genre_data'...
remote: Enumerating objects: 2953, done.
remote: Total 2953 (delta 0), reused 0 (delta 0), pack-reused 2953
Receiving objects: 100% (2953/2953), 1.62 GiB | 37.29 MiB/s, done.
Resolving deltas: 100% (34/34), done.
Checking out files: 100% (3154/3154), done.


**Check Data**

To make sure the data has been gathered.

In [44]:
!ls
%cd video_game_genre_data/
!ls

video_game_genre_data
/content/video_game_genre_data
data  extract_frame_genric.py  README.md


** Load Dependencies**

Import the keras dependencies, numpy and os files


In [0]:
# data augmenting / preprocessing
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# model architecture and training
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

# GPU check
from tensorflow.python.client import device_lib

# displaying images in notebook
from IPython.display import Image, display

# finding files
import os

# array stuff
import numpy as np

**Data Augmentation Techniques**

* Augmenting input images by making small changes, increases the number, and diversity, of input images. This will increase the model's probability of learning the correct image features

* Below follows a visual example of how images can be augmented. Keras will take care of this during training (we aren't going to use these example images during training).

In [0]:
!mkdir data/transform_samples

In [51]:
# augmentation specs
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

# original image
img = load_img('data/images/train/FPS/FPS11000.jpg')

# convert image to Numpy array (numeric value representation of image)
x = img_to_array(img)
print("Image Shape: {}".format(x.shape))

# reshape for Keras use
x = x.reshape((1,) + x.shape)
print("Image Shape: {}".format(x.shape))


# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `data/transform_samples/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='data/transform_samples', save_prefix='FPS', save_format='jpeg'):
    i += 1
    if i > 3:
        break  # otherwise the generator would loop indefinitely

Image Shape: (1080, 1920, 3)
Image Shape: (1, 1080, 1920, 3)


**Creating Model Architecture**

In [0]:
# fully connected model (opposed to model with different input / branches)
model = Sequential()

# input and first hidden layer
# - 2D images (Conv2D - https://keras.io/layers/convolutional/#conv2d)
# - 32 filters applied to input images (meaning 32 outputs per image)
# - filter/kernel has size (3x3)
# - strides is by default (1,1), meaning the filter moves one pixel at a time (both directions)
# - padding is by default 'valid', meaning if the image doesn't meet input shape, padding will be added
model.add(Conv2D(32, (3, 3), input_shape=(164, 164, 3)))
# activation function (ensures non-linearity)
model.add(Activation('relu'))
# max pooling reduces dimensionality
model.add(MaxPooling2D(pool_size=(2, 2)))

# second hidden layer
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# third hidden layer
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)
# now we need to flatten it to allow computation
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

# a normal 1D hidden layer with less neurons (not necessary)
model.add(Dense(64))
model.add(Activation('relu'))

# dropout removes influence of neurons that don't add value
model.add(Dropout(0.5))

# output layer (2 neurons) (bug with binary - now matching num of classes - https://github.com/keras-team/keras/issues/6499)
# this will give us our 'probability' value
model.add(Dense(3))

# change to a value between 0 and 1
model.add(Activation('softmax')) # binary bug (binary use 'sigmoid')

# model training configuration (https://keras.io/models/model/)
# loss function - binary crossentropy - ideal for 2 classes # bug - https://github.com/keras-team/keras/issues/6499
# optimizer - rmsprop (http://ruder.io/optimizing-gradient-descent/index.html#rmsprop) - for gradient descent (finding best weights)
# metrics - accuracy (proportion of correrctly classified images over number of total images)
model.compile(loss='categorical_crossentropy',
              optimizer='Adam', # rmsprop
              metrics=['accuracy'])

**Data Preparation / Augmentation**

In [64]:
# number of images fed into CNN at a time
# not too big - will take forever to train
# not too small - model will struggle to get a good idea of 
# the classes in general
batch_size = 500

# augment settings for training data
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255, #RGB colours (change values to 0 to 1)
        shear_range=0.2, #tilt random images (20% of images)
        zoom_range=0.2, # zoom random iamges (20% of images)
        horizontal_flip=True) # flip images

# augment settings for validation data
# this is the augmentation configuration we will use for validation:
# only rescaling
validate_datagen = ImageDataGenerator(rescale=1./255) #RGB colours (change values to 0 to 1)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/images/train',  # this is the target directory
        target_size=(164, 164),  # all images will be resized to 164x164 (same as input shape in architecture)
        batch_size=batch_size,
        class_mode='categorical')  # bug with binary - https://github.com/keras-team/keras/issues/6499- since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = validate_datagen.flow_from_directory(
        'data/images/validate',
        target_size=(164, 164),
        batch_size=batch_size,
        class_mode='categorical') # bug with 'binary' - https://github.com/keras-team/keras/issues/6499

Found 2424 images belonging to 3 classes.
Found 597 images belonging to 3 classes.


**Model Training**

In [66]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size, # total number of images processed is batch_size*steps_per_epoch*epochs
        epochs=10,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)

Epoch 1/10
4/4 [==============================] - 126s 31s/step - loss: 0.1146 - acc: 0.9662 - val_loss: 0.1155 - val_acc: 0.9560
Epoch 2/10
4/4 [==============================] - 99s 25s/step - loss: 0.1036 - acc: 0.9652 - val_loss: 0.1105 - val_acc: 0.9620
Epoch 3/10
4/4 [==============================] - 102s 25s/step - loss: 0.1132 - acc: 0.9600 - val_loss: 0.1101 - val_acc: 0.9620
Epoch 4/10
4/4 [==============================] - 95s 24s/step - loss: 0.1145 - acc: 0.9651 - val_loss: 0.1127 - val_acc: 0.9600
Epoch 5/10
4/4 [==============================] - 102s 25s/step - loss: 0.1069 - acc: 0.9670 - val_loss: 0.1158 - val_acc: 0.9620
Epoch 6/10
4/4 [==============================] - 99s 25s/step - loss: 0.1024 - acc: 0.9644 - val_loss: 0.1133 - val_acc: 0.9620
Epoch 7/10
4/4 [==============================] - 102s 25s/step - loss: 0.1046 - acc: 0.9710 - val_loss: 0.1092 - val_acc: 0.9640
Epoch 8/10
4/4 [==============================] - 99s 25s/step - loss: 0.1135 - acc: 0.9631 -

**Classifying Unseen Data**

In [69]:
batch_size = 1

test_generator = datagen.flow_from_directory(
        'data/images/test/SPT', # never seen before images - IMPORTANT - still needs subfolder(s)
        target_size=(164, 164), # same as input shape
        batch_size=batch_size, # number of images going in at the same time
        class_mode='categorical',  # this means our generator will only yield batches of data, no labels
        shuffle=False)  # our data will be in order, 5 cats then 5 dogs

# bug fix workaround - https://towardsdatascience.com/keras-a-thing-you-should-know-about-keras-if-you-plan-to-train-a-deep-learning-model-on-a-large-fdd63ce66bd2
predictions = model.predict_generator(test_generator)
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

print(predictions)

Found 52 images belonging to 1 classes.
['SPT', 'FPS', 'SPT', 'SPT', 'SPT', 'RTS', 'SPT', 'SPT', 'SPT', 'SPT', 'RTS', 'SPT', 'FPS', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'FPS', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'FPS', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'SPT', 'RTS', 'SPT', 'SPT', 'SPT', 'SPT']
